In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import flwr
import torch
import dotenv
import wandb

from torch.optim.lr_scheduler import CosineAnnealingLR

from fl_g13.architectures import BaseDino
from fl_g13.editing import SparseSGDM

from fl_g13.fl_pytorch import build_fl_dependencies

from fl_g13.fl_pytorch.datasets import reset_partition
from fl_g13.modeling import load_or_create
from fl_g13.editing import SparseSGDM, mask_dict_to_list
from torch.optim import SGD

from fl_g13.fl_pytorch.editing import load_mask

from fl_g13.fl_pytorch import get_client_app, get_server_app
from flwr.simulation import run_simulation

c:\Users\ciovi\Desktop\coding\mldl\fl-g13\.venv\lib\site-packages\dockerpycreds\utils.py:1: DeprecationWarning: The distutils package is deprecated and slated for removal in Python 3.12. Use setuptools or check PEP 632 for potential alternatives
  import distutils.spawn
2025-06-30 14:01:54.548 | INFO     | fl_g13.config:<module>:11 - PROJ_ROOT path is: C:\Users\ciovi\Desktop\coding\mldl\fl-g13


In [3]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Training on {DEVICE}")
print(f"Flower {flwr.__version__} / PyTorch {torch.__version__}")

dotenv.load_dotenv()
build_fl_dependencies()

if DEVICE == "cuda":
    backend_config = {"client_resources": {"num_cpus": 1, "num_gpus": 1.0}}
else:
    backend_config = {"client_resources": {"num_cpus": 1, "num_gpus": 0.0}}

Training on cuda
Flower 1.17.0 / PyTorch 2.7.1+cu128
'vision_transformer.py' already exists.
'utils.py' already exists.


In [4]:
# login by key in .env file
WANDB_API_KEY = dotenv.dotenv_values()["WANDB_API_KEY"]
wandb.login(key=WANDB_API_KEY)

# Load checkpoint from .env file
CHECKPOINT_DIR = dotenv.dotenv_values()["CHECKPOINT_DIR"]

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\ciovi\_netrc
wandb: Currently logged in as: tarantino-giovanbattista01 (stefano-gamba-social-politecnico-di-torino) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [5]:
# Model config
## Model Hyper-parameters
head_layers = 3
head_hidden_size = 512
dropout_rate = 0.0
unfreeze_blocks = 12

## Training Hyper-parameters
batch_size = 128
lr = 1e-3
momentum = 0.9
weight_decay = 1e-5
T_max = 8
eta_min = 1e-5

# FL config
K = 100
C = 0.1
J = 8
num_shards_per_partition = 5 # Nc
partition_type = 'shard'

num_rounds = 100

## Server App config
save_every = 5
evaluate_each = 5
fraction_fit = C        # 0.1
fraction_evaluate = C   # 0.1
min_fit_clients = 10
min_evaluate_clients = 5
min_available_clients = 10

# model editing config
model_editing = True
mask_type = 'local'
sparsity = 0.7
calibration_rounds = 3
model_editing_batch_size = 1
mask = None

## Adaptive Quorum strategy
strategy = 'quorum'
adaptive_quorum = True
# Mask option
mask_strategy = 'sum'
initial_quorum = 1
initial_target_sparsity = 0.7
# Linear mode
quorum_increment = 5
quorum_update_frequency = 5
# Adaptive mode
drift_threshold = 0.0006914556 # minimum value from the last 10 rounds of the warmup
quorum_patience = 4
force_quorum_update = 15

## simulation run config
NUM_CLIENTS = 100
MAX_PARALLEL_CLIENTS = 10

## Base model location
# The 200-epoch model folder
# Ensure that the most recent file is the correct one
model_save_path = CHECKPOINT_DIR + f"/fl/non-iid/{num_shards_per_partition}_{J}"

In [6]:
# Load Base model
model, start_epoch = load_or_create(
    path=model_save_path,
    model_class=BaseDino,
    model_config=None,
    optimizer=None,
    scheduler=None,
    device=DEVICE,
    verbose=True,
)
model.to(DEVICE)

if model_editing:
    # Create a dummy mask for SparseSGDM
    dummy_mask = [torch.ones_like(p, device=p.device) for p in model.parameters()]  
    
    optimizer = SparseSGDM(
        model.parameters(),
        mask=dummy_mask,
        lr=lr,
        momentum=momentum,
        weight_decay=weight_decay
    )
else:
    optimizer = SGD(model.parameters(), lr=lr, momentum=momentum)
    
criterion = torch.nn.CrossEntropyLoss()
scheduler = CosineAnnealingLR(
    optimizer=optimizer,
    T_max=T_max,
    eta_min=eta_min
)

# Unfreeze entire model for model_editing
model.unfreeze_blocks(unfreeze_blocks)

🔍 Loading checkpoint from /Users/ciovi/Desktop/coding/mldl/fl-g13/checkpoints/fl/non-iid/5_8\5_8_200_epoch.pth
📦 Model class in checkpoint: BaseDino
🔧 Model configuration: {'variant': 'dino_vits16', 'dropout_rate': 0.0, 'head_hidden_size': 512, 'head_layers': 3, 'num_classes': 100, 'unfreeze_blocks': 0, 'activation_fn': 'GELU', 'pretrained': True}


Using cache found in C:\Users\ciovi/.cache\torch\hub\facebookresearch_dino_main
Using cache found in C:\Users\ciovi/.cache\torch\hub\facebookresearch_dino_main


➡️ Moved model to device: cuda
✅ Loaded checkpoint from /Users/ciovi/Desktop/coding/mldl/fl-g13/checkpoints/fl/non-iid/5_8\5_8_200_epoch.pth, resuming at epoch 201


In [7]:
mask_file_name = CHECKPOINT_DIR + f'/masks/sum_{num_shards_per_partition}_{J}_{mask_type}_{sparsity}_{calibration_rounds}.pth'

sum_mask = load_mask(mask_file_name)
sum_mask = mask_dict_to_list(model, sum_mask) # converts for SparseSGDM

In [8]:
# Run simulations
reset_partition()

model_checkpoint = CHECKPOINT_DIR + f"/fl/non-iid/AdaQuo/{num_shards_per_partition}_{J}_{mask_type}_{sparsity}_{calibration_rounds}"

# Wandb settings
use_wandb = True
run_name = f"fl_adaquo_{num_shards_per_partition}_{J}_{mask_type}_{sparsity}_{calibration_rounds}"
wandb_config = {
    # wandb param
    'name': run_name,
    'project_name': f"fl_v5_{num_shards_per_partition}_{J}_adaquo",
    'run_id': run_name,
    
    # fl config
    "fraction_fit": fraction_fit,
    "lr": lr,
    "momentum": momentum,
    'weight_decay': weight_decay,
    'partition_type': partition_type,
    'K': K,
    'C': C,
    'J': J,
    'Nc': num_shards_per_partition,
    
    # model config
    'head_layers': head_layers,
    'head_hidden_size': head_hidden_size,
    'dropout_rate': dropout_rate,
    'unfreeze_blocks': unfreeze_blocks,
    
    # model editing config
    'model_editing_batch_size': model_editing_batch_size,
    'mask_calibration_round': calibration_rounds,
    'mask_type': mask_type,
    'sparsity': sparsity,
    
    # AdaQuo config
    'initial_quorum': initial_quorum,
    'initial_target_sparsity': initial_target_sparsity,
    'quorum_increment': quorum_increment,
    'quorum_update_frequency': quorum_update_frequency,
    'drift_threshold': drift_threshold,
    'quorum_patience': quorum_patience,
    'force_quorum_update': force_quorum_update
}

client = get_client_app(
    model=model,
    optimizer=optimizer,
    criterion=criterion,
    device=DEVICE,
    partition_type=partition_type,
    local_epochs=1,
    local_steps=J,
    batch_size=batch_size,
    num_shards_per_partition=num_shards_per_partition,
    scheduler=scheduler,
    model_editing=model_editing,
    mask_type=mask_type,
    sparsity=sparsity,
    mask=dummy_mask, # Will be replaced by the mask sent by the server
    model_editing_batch_size=model_editing_batch_size,
    mask_func=None,
    mask_calibration_round=calibration_rounds,
    strategy=strategy
)

server = get_server_app(
    checkpoint_dir=model_checkpoint,
    model_class=model,
    optimizer=optimizer,
    criterion=criterion,
    scheduler=scheduler,
    num_rounds=num_rounds,
    fraction_fit=fraction_fit,
    fraction_evaluate=fraction_evaluate,
    min_fit_clients=min_fit_clients,
    min_evaluate_clients=min_evaluate_clients,
    min_available_clients=min_available_clients,
    device=DEVICE,
    use_wandb=use_wandb,
    wandb_config=wandb_config,
    save_every=save_every,
    prefix='AdaQuo',
    evaluate_each=evaluate_each,
    model= model,
    start_epoch= start_epoch,
    # AdaQuo
    strategy = strategy,
    global_mask = sum_mask,
    num_total_clients = NUM_CLIENTS,
    adaptive_quorum = adaptive_quorum,
    initial_target_sparsity = initial_target_sparsity,
    quorum_increment = quorum_increment,
    quorum_update_frequency = quorum_update_frequency,
    initial_quorum = initial_quorum,
    drift_threshold = drift_threshold,
    quorum_patience = quorum_patience,
    force_quorum_update = force_quorum_update
)

# Run simulation
run_simulation(
    server_app=server,
    client_app=client,
    num_supernodes=NUM_CLIENTS,
    backend_config=backend_config
)

if use_wandb:
    wandb.finish()

Using strategy 'DynmicQuorum'


c:\Users\ciovi\Desktop\coding\mldl\fl-g13\.venv\lib\site-packages\wandb\analytics\sentry.py:259: DeprecationWarning: The `Scope.user` setter is deprecated in favor of `Scope.set_user()`.
  self.scope.user = {"email": email}  # noqa


INFO :      [DQ] ADAPTIVE mode enabled. Quorum: 53; Drift threshold: 0.0006914556
INFO :      Starting Flower ServerApp, config: num_rounds=100, no round_timeout
INFO :      
INFO :      [INIT]
INFO :      Using initial global parameters provided by strategy
INFO :      Starting evaluation of initial global parameters
Eval progress: 100%|██████████| 313/313 [00:23<00:00, 13.56batch/s]
INFO :      [Round 0] Centralized Evaluation - Loss: 1.7223, Metrics: {'centralized_accuracy': 0.5261}
INFO :      initial parameters (loss, other metrics): 1.7223209416904388, {'centralized_accuracy': 0.5261}
INFO :      
INFO :      [ROUND 1]
INFO :      [Round 1 DQ] Generated global mask with sparsity: 0.7003
INFO :      configure_fit: strategy sampled 10 clients (out of 100)
(ClientAppActor pid=4888) 2025-06-30 14:02:41.200 | INFO     | fl_g13.config:<module>:11 - PROJ_ROOT path is: C:\Users\ciovi\Desktop\coding\mldl\fl-g13
(ClientAppActor pid=4888) c:\Users\ciovi\Desktop\coding\mldl\fl-g13\.venv\lib\

💾 Saved checkpoint at: /Users/ciovi/Desktop/coding/mldl/fl-g13/checkpoints/fl/non-iid/AdaQuo/5_8_local_0.7_3\fl_AdaQuo_BaseDino_epoch_205.pth


Eval progress: 100%|██████████| 313/313 [00:22<00:00, 14.05batch/s]
INFO :      [Round 5] Centralized Evaluation - Loss: 1.4205, Metrics: {'centralized_accuracy': 0.5907}
INFO :      fit progress: (5, 1.4204622828922333, {'centralized_accuracy': 0.5907}, 402.6671535000005)
INFO :      configure_evaluate: strategy sampled 10 clients (out of 100)
INFO :      aggregate_evaluate: received 10 results and 0 failures
INFO :      
INFO :      [ROUND 6]
INFO :      configure_fit: strategy sampled 10 clients (out of 100)
INFO :      aggregate_fit: received 10 results and 0 failures
INFO :      [Round 6] Avg Drift: 0.0007 | Relative Drift: 0.0000
INFO :      fit progress: (6, {}, {}, 478.8700669000009)
INFO :      configure_evaluate: strategy sampled 10 clients (out of 100)
INFO :      aggregate_evaluate: received 10 results and 0 failures
INFO :      
INFO :      [ROUND 7]
INFO :      configure_fit: strategy sampled 10 clients (out of 100)
INFO :      aggregate_fit: received 10 results and 0 fai

...

💾 Saved checkpoint at: /Users/ciovi/Desktop/coding/mldl/fl-g13/checkpoints/fl/non-iid/AdaQuo/5_8_local_0.7_3\fl_AdaQuo_BaseDino_epoch_295.pth


Eval progress: 100%|██████████| 313/313 [00:22<00:00, 14.12batch/s]
INFO :      [Round 95] Centralized Evaluation - Loss: 0.9312, Metrics: {'centralized_accuracy': 0.7196}
INFO :      fit progress: (95, 0.9312478663822332, {'centralized_accuracy': 0.7196}, 7325.7543842)
INFO :      configure_evaluate: strategy sampled 10 clients (out of 100)
INFO :      aggregate_evaluate: received 10 results and 0 failures
INFO :      
INFO :      [ROUND 96]
INFO :      configure_fit: strategy sampled 10 clients (out of 100)
INFO :      aggregate_fit: received 10 results and 0 failures
INFO :      [Round 96] Avg Drift: 0.0005 | Relative Drift: 0.0000
INFO :      fit progress: (96, {}, {}, 7396.560510100002)
INFO :      configure_evaluate: strategy sampled 10 clients (out of 100)
INFO :      aggregate_evaluate: received 10 results and 0 failures
INFO :      
INFO :      [ROUND 97]
INFO :      configure_fit: strategy sampled 10 clients (out of 100)
INFO :      aggregate_fit: received 10 results and 0 fa

💾 Saved checkpoint at: /Users/ciovi/Desktop/coding/mldl/fl-g13/checkpoints/fl/non-iid/AdaQuo/5_8_local_0.7_3\fl_AdaQuo_BaseDino_epoch_300.pth


Eval progress: 100%|██████████| 313/313 [00:22<00:00, 14.16batch/s]
INFO :      [Round 100] Centralized Evaluation - Loss: 0.8729, Metrics: {'centralized_accuracy': 0.7397}
INFO :      fit progress: (100, 0.8728854135392954, {'centralized_accuracy': 0.7397}, 7701.5481619)
INFO :      configure_evaluate: strategy sampled 10 clients (out of 100)
INFO :      aggregate_evaluate: received 10 results and 0 failures
INFO :      
INFO :      [SUMMARY]
INFO :      Run finished 100 round(s) in 7717.76s
INFO :      	History (loss, distributed):
INFO :      		round 1: 1.347391527891159
INFO :      		round 2: 1.3935362219810485
INFO :      		round 3: 1.4075656414031983
INFO :      		round 4: 1.6614349484443665
INFO :      		round 5: 1.481391680240631
INFO :      		round 6: 1.354889065027237
INFO :      		round 7: 1.3459257066249848
INFO :      		round 8: 1.3089424788951873
INFO :      		round 9: 1.251848930120468
INFO :      		round 10: 1.4501567602157592
INFO :      		round 11: 1.1905572891235352


centralized_accuracy,▁▃▄▅▆▆▇▆▇▇▇▇▇▇█▇███▇█
centralized_eval_loss,█▆▅▄▃▃▂▃▂▂▂▁▂▂▁▂▁▁▁▁▁
decentralized_avg_eval_accuracy,▁▃▃▄▂▃▄▅▅▆▅▅▅▅▆▅▇▆▆▄▆▆▇▅▇▇▇▇▆▇▇▆▆▆▇▆▆█▆▇
decentralized_avg_eval_loss,▇█▇▆▅▄▄▅▄▄▂▅▃▃▃▂▄▂▁▃▂▂▄▃▃▂▅▂▃▃▂▂▃▃▂▂▃▁▃▁
decentralized_avg_train_accuracy,▂▃▁▄▆▆▆▅▆█▆▄▇▆▆▇▅▅▅▅▃▃▃▁▅▅▃▄▄▆▆▂▅▅▆▆▄▆▇▅
decentralized_avg_train_loss,▇█▇█▄▄▃▄▁▆▃▃▅▄▃▆▅▆▄▅▇▇▄▆▅▆▅▆▆▄▄▅▅▄▆▄▅▃▆▅
mask_sparsity,▁▁▁▁▂▂▂▂▃▃▄▄▅▅▇█████████████████████████
quorum,▁▁▁▁▁▂▂▂▂▃▄▅▅▅▆▆▇▇██████████████████████
centralized_accuracy,0.7397
centralized_eval_loss,0.87289
decentralized_avg_eval_accuracy,0.744
